In [ ]:
import os
import torch
import shutil
import random
import numpy as np
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

from tqdm import tqdm
from copy import deepcopy
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.nn import CrossEntropyLoss

import seaborn as sns
from sklearn.metrics import confusion_matrix


In [ ]:
import torchvision.models as models
from models.resnet import resnet18

In [ ]:
def get_device():
    if torch.cuda.is_available():
        device = torch.device("cuda")
    elif torch.backends.mps.is_available():
        device = torch.device("mps")
    else:
        device = torch.device("cpu")
    return device

device = get_device()
print(f"device is: {device}")

#### Create Datasets

In [ ]:
def create_dataset_directory(src_dir = "./data/test", dst_root = "./data/test"):
    os.makedirs(dst_root, exist_ok=True)
    img_exts = {'.jpg'}

    for filename in os.listdir(src_dir):
        if not any(filename.lower().endswith(ext) for ext in img_exts):
            continue 
        prefix = filename.split('_')[0]
        target_dir = os.path.join(dst_root, prefix)
        os.makedirs(target_dir, exist_ok=True)

        src_path = os.path.join(src_dir, filename)
        dst_path = os.path.join(target_dir, filename)
        
        shutil.move(src_path, dst_path)
    print("directory structure created")

create_dataset_directory(src_dir = "./data/test", dst_root = "./data/test")
create_dataset_directory(src_dir = "./data/train", dst_root = "./data/train")

In [ ]:
# increase the test dataset size. 
def move_into_test(test_dir = "data/test", train_dir = "data/train"):
    if not os.path.exists(train_dir) or not os.path.exists(test_dir):
        raise FileNotFoundError("Ensure both 'data/train' and 'data/test' directories exist.")

    for class_name in os.listdir(train_dir):
        train_class_path = os.path.join(train_dir, class_name)
        test_class_path = os.path.join(test_dir, class_name)

        if not os.path.isdir(train_class_path):
            continue  

        if not os.path.exists(test_class_path):
            os.makedirs(test_class_path)  # Create target class folder if missing

        files = os.listdir(train_class_path)
        files = [f for f in files if os.path.isfile(os.path.join(train_class_path, f))]

        if len(files) < 300:
            print(f"Skipping {class_name}: not enough files to move.")
            continue

        files_to_move = random.sample(files, 300)

        for file_name in files_to_move:
            src_path = os.path.join(train_class_path, file_name)
            dst_path = os.path.join(test_class_path, file_name)
            shutil.move(src_path, dst_path)
        print(f"moved 300 files from '{class_name}' train to test.")

# run this code only once
# move_into_test()


In [ ]:
root_train = './data/train'
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

root_test = './data/test'
test_transform = transforms.Compose([
transforms.Resize((224, 224)),
transforms.ToTensor(),
transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

test_set = datasets.ImageFolder(root_test, transform = test_transform)
train_dataset = datasets.ImageFolder(root_train, transform = train_transform)


train_size = int(0.80 * len(train_dataset))
val_size = len(train_dataset) - train_size

train_set, val_set= random_split(train_dataset, [train_size, val_size])
val_set.dataset.transform = test_transform # change transforms for val/test

In [ ]:
val_loader = DataLoader(val_set, batch_size=1024)
test_loader = DataLoader(test_set, batch_size=1024)
train_loader = DataLoader(train_set, batch_size=64, shuffle=True)

#### Utility Functions

In [ ]:
def save_checkpoint(model, checkpoint):
    torch.save(model.state_dict(), checkpoint)

def validate(model, val_loader, loss_fn = CrossEntropyLoss()):
    model.eval()
    with torch.inference_mode():
        N = 0
        mean_loss = 0
    
        for x, y in val_loader:
            x = x.to(device)
            y = y.to(device)

            outputs = model(x)
            loss = loss_fn(outputs, y)

            N += len(x)
            mean_loss += len(x) * (loss.item() - mean_loss)/N         
    return mean_loss


def accuracy(model, data_loader):
    model.eval()
    with torch.inference_mode():
      total = 0
      correct = 0
      for x, y in data_loader:
        x = x.to(device)
        y = y.to(device)

        output = model(x)
        prob = torch.softmax(output, dim = 1)
        pred = prob.max(dim = 1, keepdim = True)[1]
        correct += pred.eq(y.view_as(pred)).sum().item()
        total += len(y)
    acc = correct/total
    return acc

def plot_cm(model, test_loader, title):
    # Collect predictions and true labels for the test set
    all_preds = []
    all_labels = []
    model.eval()
    with torch.no_grad():
        for images, targets in test_loader:
            images = images.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(targets.cpu().numpy())

    # Compute the confusion matrix
    cm = confusion_matrix(all_labels, all_preds)

    # Plot the confusion matrix
    plt.figure(figsize=(10, 10))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(title)
    plt.show()
    
def calculate_precision_recall(model, dataloader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in tqdm(dataloader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)

    precision = np.sum(all_preds[all_labels == 1] == 1) / np.sum(all_preds == 1)
    recall = np.sum(all_preds[all_labels == 1] == 1) / np.sum(all_labels == 1)

    return precision, recall


#### Training Module

In [ ]:
def train(model, train_loader, val_loader, checkpoint, learning_rate = 0.01, epochs=20):
    
    loss_fn = CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    # scheduler = MultiStepLR(optimizer, milestones=[10], gamma=0.1)
    val_loss_list = []

    #acc and loss before training
    val_loss = validate(model, val_loader)
    val_loss_list.append(val_loss)
    best_loss = val_loss
    best_model = deepcopy(model)
    print(f"validation loss prior to training: {val_loss:.4f}")
    
    for epoch in tqdm(range(0, epochs)):
        model.train()
        for x, y in train_loader:
            x = x.to(device)
            y = y.to(device)

            outputs = model(x)
            loss = loss_fn(outputs, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        # scheduler.step()
        #acc and loss 
        model.eval()
        val_loss = validate(model, val_loader)

        val_loss_list.append(val_loss)

        if best_loss > val_loss:
            best_loss = val_loss
            best_model = deepcopy(model)
            save_checkpoint(model, checkpoint)
        print(f"val loss : {val_loss:.4f}")

    return best_model, val_loss_list


#### Experiment with ResNet18

In [ ]:
def get_resnet18(num_classes = 29, pretrained=False):
    model = resnet18(pretrained)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    model = model.to(device)
    return model
model = get_resnet18(num_classes=29, pretrained=False)
model_path = 'resnet18.pth' 
if os.path.exists(model_path):
    model.load_state_dict(torch.load(model_path, map_location='cpu'))
    model = model.to(device)
else:
    model, val_loss_list = train(model, train_loader, val_loader, 'resnet18.pth')
    np.save('val_loss.npy', val_loss_list)
    save_checkpoint(model, 'resnet18.pth')

#### Visualize loss curve

In [ ]:
val_loss_list = np.load('resnet18_val_loss.npy')
plt.plot(val_loss_list, label='val Loss')
plt.legend()
plt.title("training")
plt.show()

#### visualize features

In [ ]:
def visualize_feature_maps(model, layer, image_tensor):
    def hook_fn(module, input, output):
        plt.figure(figsize=(12, 12))
        for i in range(min(64, output.size(1))):
            plt.subplot(8, 8, i + 1)
            plt.imshow(output[0, i].detach().cpu().numpy(), cmap='gray')
            plt.axis('off')
        plt.suptitle("Feature Maps")
        plt.show()

    handle = layer.register_forward_hook(hook_fn)
    _ = model(image_tensor.unsqueeze(0).to(device))
    handle.remove()

image, _ = test_set[0]
visualize_feature_maps(model, model.conv1, image)

Calculate Precision/Recall

In [ ]:

precision, recall = calculate_precision_recall(model, test_loader)
print(f"Precision: {precision:.4f}, Recall: {recall:.4f}")

Plot confusion Matrix

In [ ]:
plot_cm(model, test_loader, title = "Resnet18 Confusion Matrix")


#### evaluate trained model

In [ ]:
test_accuracy = accuracy(model, test_loader)
print(f"test accuracy of trained model is: {test_accuracy*100:.3f}")

In [ ]:
val_accuracy = accuracy(model, val_loader)
print(f"val accuracy of trained model is: {val_accuracy*100:.3f}")

#### Experiment with DenseNet101

In [ ]:
def get_densenet121(num_classes = 29, pretrained=False):
    model = models.densenet121(pretrained=pretrained)
    model.classifier = nn.Linear(model.classifier.in_features, num_classes)
    model = model.to(device)
    return model
model = get_densenet121(num_classes=29, pretrained=True)
model_path = 'densenet121.pth' 
if os.path.exists(model_path):
    model.load_state_dict(torch.load(model_path, map_location='cpu'))
else:
    model, val_loss_list = train(model, train_loader, val_loader, model_path)
    np.save('densenet121_val_loss.npy', val_loss_list)
    save_checkpoint(model, model_path)

In [ ]:
val_loss_list = np.load('densenet121_val_loss.npy')
plt.plot(val_loss_list, label='val Loss')
plt.legend()
plt.title("training")
plt.show()

In [ ]:
for name, module in model.named_modules():
    if isinstance(module, nn.Conv2d):
        print(name, module)

In [ ]:
image, _ = test_set[0]
visualize_feature_maps(model, model.features.denseblock1.denselayer1.conv1, image)

In [ ]:
test_accuracy = accuracy(model, test_loader)
print(f"test accuracy of trained model is: {test_accuracy*100:.3f}")
val_accuracy = accuracy(model, val_loader)
print(f"val accuracy of trained model is: {val_accuracy*100:.3f}")

In [ ]:
print(f"Precision: {precision:.4f}, Recall: {recall:.4f}")
plot_cm(model, test_loader, title = "Densenet121 Confusion Matrix")